<h1>Sentiment Analysis using a pre-trained model for Aspect-Based Sentiment Analysis (ABSA)</h1>
Inspired by <a href='https://www.kaggle.com/code/bekirztrk/restaurant-absa-using-pre-trained-model-nlp'>this kaggle project</a>, I'll be running the restaurant reviews I've collected through the pre-trained DeBERTa-v3 model, documented <a href='https://huggingface.co/yangheng/deberta-v3-base-absa-v1.1'>here</a>.

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F


tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

c:\Users\andy-\anaconda3\envs\belleviz\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\andy-\.cache\huggingface\hub\models--yangheng--deberta-v3-base-absa-v1.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

c:\Users\andy-\anaconda3\envs\belleviz\lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [2]:
def analyze_aspect_sentiment(review, aspects, threshold=0.9):
    results = {}
    for aspect in aspects:
        # first check if aspect is mentioned in the review
        if aspect.lower() in review.lower():
            inputs = tokenizer(f"[CLS] {review} [SEP] {aspect} [SEP]", return_tensors="pt")
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            probs = probs.detach().numpy()[0]

            # extract label with highest probability
            max_label, max_prob = max(zip(["negative", "neutral", "positive"], probs), key=lambda x: x[1])

            if max_prob >= threshold:
                results[aspect] = (max_label, max_prob)
            
            # print(f"Sentiment of aspect '{aspect}' in the review is {max_label} with probability {max_prob:.4f}\n")
    return results

In [3]:
# Load data
import pandas as pd

df = pd.read_csv('reviews.csv')

name_list = df['name'].tolist()
review_list = df['text'].tolist()


In [ ]:
# List of aspects to analyze
aspects_to_analyze = ["food", "service", "ambience",
                      "staff", "price", "value", "restaurant",
                      "recommendation", "parking", "drink",
                      "expectation", "flavor"]

# Set the probability threshold
threshold = 0.1

# Analyze sentiment for each aspect in each review
from tqdm.notebook import tqdm
all_results = {}
for review in tqdm(review_list):
    results = analyze_aspect_sentiment(review, aspects_to_analyze, threshold)
    all_results[review] = results

  0%|          | 0/936 [00:00<?, ?it/s]

In [ ]:
# Map results to the dataframe
df['results'] = df['text'].map(all_results)

In [ ]:
# Write to CSV
df.to_csv("lower_threshold.csv", index=False)

In [ ]:
absa_columns = ["food", "service", "ambience",
                "staff", "price", "value", "restaurant",
                "recommendation", "parking", "drink",
                "expectation", "flavor"]
df[absa_columns] = df[absa_columns].applymap(sentiment_to_score)
df[absa_columns] = df[absa_columns].applymap(dec_to_int_score)

C:\Users\andy-\AppData\Local\Temp\ipykernel_28352\3526763907.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[absa_columns] = df[absa_columns].applymap(sentiment_to_score)
C:\Users\andy-\AppData\Local\Temp\ipykernel_28352\3526763907.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[absa_columns] = df[absa_columns].applymap(dec_to_int_score)
